## Topographische Modelle für UIW (120.109)

### Protokoll 2

### Präambel

Fügen Sie Ihren Code bzw. ihre Antworten an den Stellen ein, die mit `YOUR CODE HERE` oder \"YOUR ANSWER HERE\" gekennzeichnet sind. **Wichtig**: Bevor Sie die Aufgabe abgeben, stellen Sie sicher, dass die `raise NotImplementedError()` Zeilen aus Ihrem Code entfernt wurden, und dass Ihr Code fehlerlos durchläuft. Starten Sie dazu den Kernel neu und führen Sie alle Zellen aus (Doppelpfeilsymbol in der Menüleiste).

### Aufgabe 1
Gegeben ist ein Set an (synthetischen) Punkten in der Datei *punkte.xyz*. Eine 3D-Ansicht der Punktwolke ist in der folgenden Abbildung zu sehen:

![Synthetische 3D-Punkte](3d-points.png)

Die Punkte umspannen im Grundriss ein Quadrat von 10m Seitenlänge, und der Datensatz weist eine minimale Höhe von 1m (blaue Punkte) und zwei Spitzen mit einer Höhe von bis zu 7.75m (rote Punkte) auf.

Lesen Sie zunächst die Punkte mittels [Import](https://opals.geo.tuwien.ac.at/html/stable/ModuleImport.html) in einen OPALS Datenmanager ein und speichern Sie diesen in der Datei *punkte.odm*. Interpolieren Sie anschließend mittels [Grid](https://opals.geo.tuwien.ac.at/html/stable/ModuleGrid.html) verschiedene Flächenmodelle nach den u.a. Vorgaben und speichern Sie diese jeweils als regelmäßges Raster mit einer Gitterweite von 2cm. Leiten Sie für jedes Gittermodell mit [Shade](https://opals.geo.tuwien.ac.at/html/stable/ModuleShade.html) eine Schummerung und mit [ZColor](https://opals.geo.tuwien.ac.at/html/stable/ModuleZColor.html) eine Höhenkodierung ab. Verwenden Sie für die Benennung der Ausgabedateien die in der kommentierten Codezelle angegebenen Dateinamen. Erzeugen Sie vier Modelle mit den folgenden Interpolationsvarianten:

- Delaunay Triangulierung (*delaunayTriangulation*)
- Gleitende Schrägebene (*movingPlanes*) mit n gleichgewichteten Nachbarpunkten 
- Gleitendes Mittel (*movingAverage*) mit n Nachbarpunkten und inverser Distanzgewichtung: $w(d) = \frac{1}{1+\alpha d^\beta}$
- Lineare Prädiktion/Kriginng (*kriging*)

Anmekungen und Hinweise:
- Verwenden Sie in allen Fällen einen Suchradius von 15m, der sicher alle Punkte einschließt.
- Wählen Sie *n* (Anzahl der Nachbarpunkte) für die *movingPlanes* und die *movingAverage* Interpolation so, dass die Überbestimmung mindestens 1 beträgt. 
- Setzen *n* für die kriging Interpolation gleich der Gesamtpunktanzahl (=39).
- Verwenden Sie für die Höhenkodierung eine Äqudistanzintervall (*interval*) von 10cm und eine Skalierung der (Standard)Farbpallette (*scalePal*) von  0-8. Durch die Verwendung identischer Visualisierungsparameter können Sie im Nachgang die Ergebnisse besser unter einander vergleichen.
- Bei der Verwendung der gleitenden Schrägenen- bzw Mittel-Interpolation wird es aufgrund des relativ zur Gitterweite großen Stützpunktabstandes zu Höhensprüngen kommen, wann immer sich die Nachbarschaft ändert. Beziehen Sie das bei der Beurteilung der Ergebnisse mit ein. 
- ad *movingAverage*: Experimentieren Sie mit der Gewichtsfunktion ([weightFunc](https://opals.geo.tuwien.ac.at/html/stable/ModuleGrid.html#grid_example_5)), um einen möglichst glatten Verlauf der interpolierten Gitterpunktshöhen zu erreichen. Ein glatter Übergang wird immer dann erreicht, wenn bei Änderung der Nachbarschaft, die wegfallenden bzw. neu hinzukommenden Punkte mit einem Gewicht nahe Null aufweisen.  Wählen Sie dazu geeignete Werte für $\alpha$ und $\beta$ (siehe Codezelle weiter unten) sowie für die Nachbarnanzahl. Hinweis: Ein perfekter Übergang wird sich bei der vorliegenden Datenlage/Aufgabenstellung  nicht erreichen lassen (siehe Schummerung).

In [1]:
#
# Import Statements
#
import opals
from opals import Import, Grid, Shade, ZColor

In [2]:
###################################################################
# Überprüfung/Sicherstellung der Nicht-Existenz der Ergbnisdateien
###################################################################
from os.path import exists
from os import remove

def del_if_exists(fp):
    if exists(fp):
        remove(fp)
        
# Variablendefinition (Dateinamen)
odm = 'punkte.odm' 

tin_mod = 'tin.tif'            # Dateiname für Rastermodell aus Delaunay Triangulation
mpl_mod = 'mpl.tif'            # Dateiname für Rastermodell aus Moving Planes Interpolation
mav_mod = 'mav.tif'            # Dateiname für Rastermodell aus Moving Average Interpolation
kri_mod = 'kri.tif'            # Dateiname für Rastermodell aus Lineare Prädiktion / Kriging
#
tin_shd = 'tin_shd.png'        # Dateiname für Schummerung von tin_mod
mpl_shd = 'mpl_shd.png'        # Dateiname für Schummerung von mpl_mod
mav_shd = 'mav_shd.png'        # Dateiname für Schummerung von mav_mod
kri_shd = 'kri_shd.png'        # Dateiname für Schummerung von kri_mod
#
tin_zco = 'tin_zco.png'        # Dateiname für Höhenkodierung von tin_mod
mpl_zco = 'mpl_zco.png'        # Dateiname für Höhenkodierung von mpl_mod
mav_zco = 'mav_zco.png'        # Dateiname für Höhenkodierung von mav_mod
kri_zco = 'kri_zco.png'        # Dateiname für Höhenkodierung von kri_mod

# Dateien löschen
del_if_exists(odm)
#
del_if_exists(tin_mod)
del_if_exists(mpl_mod)
del_if_exists(mav_mod)
del_if_exists(kri_mod)
#
del_if_exists(tin_shd)
del_if_exists(mpl_shd)
del_if_exists(mav_shd)
del_if_exists(kri_shd)
#
del_if_exists(tin_zco)
del_if_exists(mpl_zco)
del_if_exists(mav_zco)
del_if_exists(kri_zco)
#

In [3]:
# Importieren der Punkte
# YOUR CODE HERE
Import.Import(inFile = 'punkte.xyz', outFile = odm).run()

[Parameter(path=inFile, values=punkte.xyz, type=Vector<VectorOrRasterFile>, source=interFace),
 Parameter(path=iFormat, values=xyz, type=Vector<String>, source=estimated),
 Parameter(path=trjFile, values=, type=Vector<Path>, source=unknown),
 Parameter(path=tFormat, values=, type=Vector<String>, source=estimated),
 Parameter(path=outFile, values=punkte.odm, type=Path, source=interFace),
 Parameter(path=filter, values=, type=FilterWithPlaceHolders<coordinatesAttributes, coordinates | attributes | delayedResults>, source=unknown),
 Parameter(path=trafo, values=, type=TrafPars3dAffine, source=unknown),
 Parameter(path=storeOrder, values=, type=bool, source=unknown),
 Parameter(path=tilePointCount, values=200000, type=int32, source=internalDefault),
 Parameter(path=tileSize, values=, type=double, source=unknown),
 Parameter(path=storeBeamInfo, values=BeamVector, type=Vector<BeamInfo>, source=internalDefault),
 Parameter(path=mounting, values=, type=MountingPars, source=unknown),
 Parameter

In [4]:
#
# (Gewichts)funktion für inverse Distanzgewichtung bei der movingAverage Interpolation
#
alpha, beta = 3, 5                                             # Wälen Sie geeignete Werte für alpha und beta
wf= "1. / ( 1 + {:} * pow(Dist2D(n[0],n[1]),{:}))".format(alpha,beta)

#
# Setzten von weiteren allgemeinen Parametern (Gitterweite, Suchradius, ZColor-Parameter, etc.)
#
# YOUR CODE HERE
gitterweite = 0.02
suchradius = 15
scaleP='0 8'
palF = 'standardPal.xml'

In [5]:
# Berechnung und Visualisierung der Modellvariante: Dreiecksvermaschung (delaunayTriangulation)
# YOUR CODE HERE
grd = Grid.Grid(inFile = odm, outFile = tin_mod)
grd.gridSize = gitterweite
grd.searchRadius = suchradius
grd.selMode = opals.Types.SelectionMode.quadrant
grd.interpolation = opals.Types.GridInterpolator.delaunayTriangulation
grd.run()

Shade.Shade(inFile=tin_mod, outFile=tin_shd, sunPosition=[350,50],shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile=tin_mod, outFile=tin_zco, interval=0.1, palFile = palF, scalePal = scaleP).run()

[Parameter(path=inFile, values=tin.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=tin_zco.png, type=Path, source=interFace),
 Parameter(path=oFormat, values=PNG, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.0199999996, type=float, source=estimated),
 Parameter(path=interval, values=0.100000001, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=tin_zcoLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=0 8, type=ZColorScalePal, source=interFace),
 Parameter(path=offsetPal, values=, type=float, source=unknown),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=

In [48]:
# Berechnung und Visualisierung der Modellvariante: Gleitende Schrägebene (movingPlanes)
# YOUR CODE HERE
grd = Grid.Grid(inFile = odm, outFile = mpl_mod)
grd.gridSize = gitterweite
grd.searchRadius = suchradius
grd.neighbours = 6
grd.selMode = opals.Types.SelectionMode.quadrant
grd.interpolation = opals.Types.GridInterpolator.movingPlanes
grd.run()

Shade.Shade(inFile=mpl_mod, outFile=mpl_shd, sunPosition=[350,50],shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile=mpl_mod, outFile=mpl_zco, interval=0.1, palFile = palF, scalePal = scaleP).run()

[Parameter(path=inFile, values=mpl.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=mpl_zco.png, type=Path, source=interFace),
 Parameter(path=oFormat, values=PNG, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.0199999996, type=float, source=estimated),
 Parameter(path=interval, values=0.100000001, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=mpl_zcoLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=0 8, type=ZColorScalePal, source=interFace),
 Parameter(path=offsetPal, values=, type=float, source=unknown),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=

In [49]:
# Berechnung und Visualisierung der Modellvariante: Gleitendes Mittel (movingAverage)
# YOUR CODE HERE
grd = Grid.Grid(inFile = odm, outFile = mav_mod)
grd.gridSize = gitterweite
grd.searchRadius = suchradius
grd.neighbours = 8
grd.selMode = opals.Types.SelectionMode.quadrant
grd.interpolation = opals.Types.GridInterpolator.movingAverage
grd.weightFunc = wf
grd.run()

Shade.Shade(inFile=mav_mod, outFile=mav_shd, sunPosition=[350,50],shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile=mav_mod, outFile=mav_zco, interval=0.1, palFile = palF, scalePal = scaleP).run()

[Parameter(path=inFile, values=mav.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=mav_zco.png, type=Path, source=interFace),
 Parameter(path=oFormat, values=PNG, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.0199999996, type=float, source=estimated),
 Parameter(path=interval, values=0.100000001, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=mav_zcoLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=0 8, type=ZColorScalePal, source=interFace),
 Parameter(path=offsetPal, values=, type=float, source=unknown),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=

In [50]:
# Berechnung und Visualisierung der Modellvariante: Lineare Prädiktion (kriging)
# YOUR CODE HERE
grd = Grid.Grid(inFile = odm, outFile = kri_mod)
grd.gridSize = gitterweite
grd.searchRadius = suchradius
grd.neighbours = 39
grd.selMode = opals.Types.SelectionMode.quadrant
grd.interpolation = opals.Types.GridInterpolator.kriging
grd.run()

Shade.Shade(inFile=kri_mod, outFile=kri_shd, sunPosition=[350,50],shading = opals.Types.ShadingMethod.tanaka).run()

ZColor.ZColor(inFile=kri_mod, outFile=kri_zco, interval=0.1, palFile = palF, scalePal = scaleP).run()

[Parameter(path=inFile, values=kri.tif, type=Path, source=interFace),
 Parameter(path=outFile, values=kri_zco.png, type=Path, source=interFace),
 Parameter(path=oFormat, values=PNG, type=String, source=estimated),
 Parameter(path=pixelSize, values=0.0199999996, type=float, source=estimated),
 Parameter(path=interval, values=0.100000001, type=Vector<float>, source=interFace),
 Parameter(path=nClasses, values=, type=uint32, source=unknown),
 Parameter(path=palFile, values=standardPal.xml, type=Path, source=interFace),
 Parameter(path=legend, values=file=kri_zcoLegend.svg, type=ZColorLegend, source=estimated),
 Parameter(path=scalePal, values=0 8, type=ZColorScalePal, source=interFace),
 Parameter(path=offsetPal, values=, type=float, source=unknown),
 Parameter(path=zRange, values=, type=Array<float,2>, source=unknown),
 Parameter(path=limit, values=, type=GridLimit, source=unknown),
 Parameter(path=resampling, values=, type=ResamplingMethod, source=unknown),
 Parameter(path=band, values=

In [51]:
# Sichtbare Überprüfungen
assert exists(odm)
assert exists(tin_mod)
assert exists(mpl_mod)
assert exists(mav_mod)
assert exists(kri_mod)
assert exists(tin_shd)
assert exists(mpl_shd)
assert exists(mav_shd)
assert exists(kri_shd)
assert exists(tin_zco)
assert exists(mpl_zco)
assert exists(mav_zco)
assert exists(kri_zco)
print('Alle Ergebnisdateien vorhanden.')

Alle Ergebnisdateien vorhanden.


In [52]:
# Verborgene Überprüfungen 1 (delaunayTriangulation)

In [53]:
# Verborgene Überprüfungen 2 (movingPlanes)

In [54]:
# Verborgene Überprüfungen 3 (movingAverage)

In [55]:
# Verborgene Überprüfungen 4 (kriging)

### Visualisierung der Ergebnisse

#### Delaunay Triangulierung
![tin_shd.png](tin_shd.png)
![tin_zco.png](tin_zco.png)

#### Gleitende Schrägebene
![mpl_shd.png](mpl_shd.png)
![mpl_zco.png](mpl_zco.png)

#### Gleitendes Mittel (mit inv. Distanzgewichtung)
![mav_shd.png](mav_shd.png)
![mav_zco.png](mav_zco.png)

#### Lineare Prädiktion / Kriging
![kri_shd.png](kri_shd.png)
![kri_zco.png](kri_zco.png)




#### Diskussion
Interpretieren Sie die Ergebnisse und fassen Sie die wesentlichen Unterschiede und Eigenschaften der verwendeten Interpolationsmethoden in wenigen Sätzen zusammen. Sichworte:
- Einfluss der Nachbarschaft
- Stetigkeit der Höhen und Neigungen (Glattheit)
- lokale vs globale Interpolationsmethoden

Besonders ausgeprägte Höhenunterschiede und somit große Steigungen weisen sowohl die Interpolation mittels Delaunay Triangulierung als auch die mittels Gleitendem Mittel (mit inverser Distanzgewichtung) auf. Das Gleitende Mittel hat dabei aber noch einen durchgehend stetigen Verlauf, während die Triangulierung bei größeren Ausreißern riesige Gradienten bekommen kann (großer Einfluss der Nachbarschaft).

Stetig ist auch das Modell der globalen Kriging-Interpolation, wobei hier die Extrema deutlich abgeschwächt sind, was eine fast schon zu starke Glättung erzeugt.

Eindeutig unstetig zeigt sich das Modell der Gleitenden Schrägebene. Das liegt jedoch an der Methode selbst. Die Höhenunterschiede sind etwas geringer als bei Delaunay und beim Gleitenden Mittel und es gibt sehr raue Stellen.

Insgesamt würde ich in diesem Beispiel der Interpolation mittels Gleitendem Mittel die beste Note ausstellen, da sie das anschaulichste Geländemodell erzeugt, dass gleichzeitig stetig ist, aber auch die Maximalwerte gut darstellt.